In [1]:
import numpy as np
import pandas as pd
# For visualizations
import matplotlib.pyplot as plt
# For regular expressions
import re
# For handling string
import string
# For performing mathematical operations
import math
import os

from collections import Counter
from langdetect import detect


import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


nltk.download('stopwords')
pd.options.mode.chained_assignment = None

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hudsb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train = pd.read_csv('analysis/Train.csv') 
test = pd.read_csv('analysis/Test.csv')
print(train.text.apply(str))
test.text.apply(str)
train.head()

0        I grew up (b. 1965) watching and loving the Th...
1        When I put this movie in my DVD player, and sa...
2        Why do people who do not know what a particula...
3        Even though I have great interest in Biblical ...
4        Im a die hard Dads Army fan and nothing will e...
                               ...                        
39995    "Western Union" is something of a forgotten cl...
39996    This movie is an incredible piece of work. It ...
39997    My wife and I watched this movie because we pl...
39998    When I first watched Flatliners, I was amazed....
39999    Why would this film be so good, but only gross...
Name: text, Length: 40000, dtype: object


,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


### olhar funções do outro notebook (é o mesmo processo de limpeza):

In [3]:
# Dictionary of English Contractions
contractions_dict = { 
                     "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not", "couldn`t": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have", "Ive": "I have", "Im": "I am", "i'd": "I do", 
                     "thats": "that is", "i'm": "I am", "You‘re": "you are", "That’s": "that is", "I’m": "I am", "/": " "}

# Function for expanding contractions
def expand_contractions(self, contractions_dict=contractions_dict):

# Regular expression for finding contractions
    contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, self)
## chat words
chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""
chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

def lower_text(self):
    """
    Make text lowercase
    """
    return self.lower()

def remove_stopwords(self):
    
    stop_words = set(nltk.corpus.stopwords.words('english'))
    #não remover palavras importantes
    exclude_words = set(("or", "no"))
    new_stop_words = stop_words.difference(exclude_words)
    """custom function to remove the stopwords"""
    return " ".join([word.lower() for word in str(self).split() if word not in new_stop_words])

def remove_symbols(text):   
    
    #remover @username 
    text = re.sub(r'@([^\s]+)', r'', text)
    
    #remover pontuação
    PUNCT_TO_REMOVE = string.punctuation
    text = text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    
    #remover www.* or https?://* 
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ', text)
   
  
    #substituir #word para word(remover hashtag)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    
    #remover parentes e conteudo dos parentesis 
    "usar ou não????"
    text = re.sub(r'\([^)]*\)', '', text)
    
    #remover todos os outros caracteres (exceto numeros e letras)
    text = re.sub(r"[^\w\s]", '', text)
    
    #substituir traço por espaço:
    text = re.sub(r"([a-z])\-([a-z])", r"\1 \2", text , 0, re.IGNORECASE)

    #trim
    text = text.strip('\'"')
    
    #remover letras em excesso, como happyyyyyyyy fica happyy:
    "precisa usar?"
    rpt_regex = re.compile(r"(.)\1{1,}", re.IGNORECASE)
    text = rpt_regex.sub(r"\1\1", text)
    
    #codigo dos emojis:
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
    
    #remover emojis do texto, no futuro podemos substiuir por palavras:
    def emoji(text):
        text = emoji_pattern.sub(r'', text)
        return text
    
    #remover emoticons:
    emoticons = \
    [
     ('__positive__',[ ':-)', ':)', '(:', '(-:', \
                       ':-D', ':D', 'X-D', 'XD', 'xD', \
                       '<3', ':\*', ';-)', ';)', ';-D', ';D', '(;', '(-;', ] ),\
     ('__negative__', [':-(', ':(', '(:', '(-:', ':,(',\
                       ':\'(', ':"(', ':((', ] ),\
    ]
    
    def replace_parenth(arr):
        return [text.replace(')', '[)}\]]').replace('(', '[({\[]') for text in arr]
    
    def regex_join(arr):
        return '(' + '|'.join( arr ) + ')'

    emoticons_regex = [ (repl, re.compile(regex_join(replace_parenth(regx))) ) \
            for (repl, regx) in emoticons ]
    
    for (repl, regx) in emoticons_regex :
        text = re.sub(regx, ' '+repl+' ', text)
    
    return text

def tokenizer(text):
    word_list = nltk.word_tokenize(text)
    return word_list

def lemmatize_words(text):
    lemmatizer = WordNetLemmatizer()
    wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
    pos_tagged_text = nltk.pos_tag(text.split())
    sentence_words = tokenizer(text)
    for word in sentence_words:
        sentence_words.remove(word)
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
    

def cleaning_process(df):
    
    df['no_contractions'] = df['text'].apply(expand_contractions) 
    df['no_chat'] = df['no_contractions'].apply(chat_words_conversion)
    df['lower_text'] = df['no_chat'].apply(lower_text)   
    df['no_stop'] = df['lower_text'].apply(remove_stopwords)
    df['no_symbols'] = df['no_stop'].apply(remove_symbols) 
    df['lemma'] = df['no_symbols'].apply(lemmatize_words)
    df['token'] = df['lemma'].apply(tokenizer)



    df = df.drop(['no_contractions', 'no_chat', 'lower_text', 'no_stop'], axis='columns', inplace=True)
    return df  
    



In [4]:
cleaning_process(train)
train.head()

,text,label,no_symbols,lemma,token
0,I grew up (b. 1965) watching and loving the Th...,0,grew b 1965 watching loving thunderbirds mates...,grow b 1965 watch love thunderbird mat school ...,"[grow, b, 1965, watch, love, thunderbird, mat,..."
1,"When I put this movie in my DVD player, and sa...",0,put movie dvd player sat coke chips expectatio...,put movie dvd player sit coke chip expectation...,"[put, movie, dvd, player, sit, coke, chip, exp..."
2,Why do people who do not know what a particula...,0,people know particular time past like feel nee...,people know particular time past like feel nee...,"[people, know, particular, time, past, like, f..."
3,Even though I have great interest in Biblical ...,0,even though great interest biblical movies bor...,even though great interest biblical movie bore...,"[even, though, great, interest, biblical, movi..."
4,Im a die hard Dads Army fan and nothing will e...,1,die hard dads army fan nothing ever change tha...,die hard dad army fan nothing ever change that...,"[die, hard, dad, army, fan, nothing, ever, cha..."


## spellchecker
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

## chat words
chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""
chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import pickle


def save_pkl_pickle(model_path):
    pickle.dump(model, open(model_path+"model_teste.pickle", 'wb'))
    pickle.dump(vectorizer_model, open(model_path+"vectorizer_teste.pickle", "wb"))
    print ("====done saving into pickle using Pickle!====")


def train_test_model(X, y):
    #Vectorizer
    #default tokenization and preprocessing method is applied unless other functions are specified
    vectorizer_model = TfidfVectorizer(stop_words='english', lowercase=False, max_features=2000)
    #The astype(‘U’) is telling numpy to convert the data to Unicode (essentially a string in python 3). https://numpy.org/devdocs/reference/arrays.dtypes.html for reference.
    X = vectorizer_model.fit_transform(X.values.astype('U'))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

    # MODEL Training
    model = LogisticRegression(max_iter=200)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("Training Accuracy :", model.score(X_train, y_train))
    print("Validation Accuracy :", model.score(X_test, y_test))
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
    
    # confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print("Confurion Matrix:", cm)
    
    return model, vectorizer_model

Project_path = "analysis"
model_path = Project_path + "/models/"

model, vectorizer_model = train_test_model(train["lemma"], train["label"])
save_pkl_pickle(model_path)


Training Accuracy : 0.8876875
Validation Accuracy : 0.867875
Accuracy: 0.867875
Confurion Matrix: [[3415  583]
 [ 474 3528]]
====done saving into pickle using Pickle!====


In [7]:
cleaning_process(test)
test.head()

,text,label,no_symbols,lemma,token
0,I always wrote this series off as being a comp...,0,always wrote series complete stinkfest jim bel...,always write series complete stinkfest jim bel...,"[always, write, series, complete, stinkfest, j..."
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0,1st watched 12 7 2002 3 10dirsteve purcell ty...,1st watch 12 7 2002 3 10dirsteve purcell typic...,"[1st, watch, 12, 7, 2002, 3, 10dirsteve, purce..."
2,This movie was so poorly written and directed ...,0,movie poorly written directed fell asleep 30 m...,movie poorly write direct fell asleep 30 minut...,"[movie, poorly, write, direct, fell, asleep, 3..."
3,The most interesting thing about Miryang (Secr...,1,interesting thing miryang secret sunshine acto...,interesting thing miryang secret sunshine acto...,"[interesting, thing, miryang, secret, sunshine..."
4,"when i first read about ""berlin am meer"" i did...",0,first read berlin meer expect much thought rig...,first read berlin meer expect much think right...,"[first, read, berlin, meer, expect, much, thin..."


In [9]:
def loading_pickle(model_path):
    vectorizer = pickle.load(open(model_path+"vectorizer_teste.pickle",'rb'))
    model = pickle.load(open(model_path+"model_teste.pickle",'rb'))
    return vectorizer, model


def predict(model, vectorizer, self):
    pred = model.predict(vectorizer.transform(self))
    print("predicted class:", pred)
    return pred

vectorizer, model = loading_pickle(model_path)
predict = predict(model, vectorizer, test.lemma)


predicted class: [1 0 0 ... 0 0 0]


In [10]:
test["sentiment"] = predict

In [11]:
test.head(50)

,text,label,no_symbols,lemma,token,sentiment
0,I always wrote this series off as being a comp...,0,always wrote series complete stinkfest jim bel...,always write series complete stinkfest jim bel...,"[always, write, series, complete, stinkfest, j...",1
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0,1st watched 12 7 2002 3 10dirsteve purcell ty...,1st watch 12 7 2002 3 10dirsteve purcell typic...,"[1st, watch, 12, 7, 2002, 3, 10dirsteve, purce...",0
2,This movie was so poorly written and directed ...,0,movie poorly written directed fell asleep 30 m...,movie poorly write direct fell asleep 30 minut...,"[movie, poorly, write, direct, fell, asleep, 3...",0
3,The most interesting thing about Miryang (Secr...,1,interesting thing miryang secret sunshine acto...,interesting thing miryang secret sunshine acto...,"[interesting, thing, miryang, secret, sunshine...",0
4,"when i first read about ""berlin am meer"" i did...",0,first read berlin meer expect much thought rig...,first read berlin meer expect much think right...,"[first, read, berlin, meer, expect, much, thin...",0
5,"I saw this film on September 1st, 2005 in Indi...",1,saw film september 1st 2005 indianapolis one j...,saw film september 1st 2005 indianapolis one j...,"[saw, film, september, 1st, 2005, indianapolis...",1
6,I saw a screening of this movie last night. I ...,0,saw screening movie last night high expectatio...,saw screen movie last night high expectation g...,"[saw, screen, movie, last, night, high, expect...",1
7,William Hurt may not be an American matinee id...,1,william hurt may american matinee idol anymore...,william hurt may american matinee idol anymore...,"[william, hurt, may, american, matinee, idol, ...",0
8,IT IS A PIECE OF CRAP! not funny at all. durin...,0,piece crap funny all whole movie nothing ever ...,piece crap funny all whole movie nothing ever ...,"[piece, crap, funny, all, whole, movie, nothin...",0
9,I'M BOUT IT(1997)<br /><br />Developed & publi...,0,im bout it1997br br developed published no li...,im bout it1997br br develop publish no limit f...,"[im, bout, it1997br, br, develop, publish, no,...",0


In [14]:
X, y = train['text'], train['label']
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.20)


In [15]:
from sklearn.pipeline import Pipeline# Visualisation


pipe = Pipeline([('vectoriser', 
                  TfidfVectorizer(min_df=30, 
                                  max_df=.6, 
                                  ngram_range=(1,2))),
                 ('model', 
                   LogisticRegression(max_iter=200)
                                                    )])

pipe.fit(X_train, y_train)

Pipeline(steps=[('vectoriser',
                 TfidfVectorizer(max_df=0.6, min_df=30, ngram_range=(1, 2))),
                ('model', LogisticRegression(max_iter=200))])

In [17]:

############# THE COEFICIENTS OF OUR PIPELINE 

coefs = pd.DataFrame(pipe['model'].coef_, 
                     columns=pipe['vectoriser'].get_feature_names())

coefs = coefs.T.rename(columns={0:'coef'}).sort_values('coef')


#### CHECK RESULTS
print(coefs.head(20))
print('\n')
print(coefs.tail(20))

               coef
bad       -8.580168
worst     -7.898349
awful     -6.538099
the worst -6.087067
boring    -5.865988
waste     -5.774631
poor      -5.525120
terrible  -5.256204
nothing   -4.547524
horrible  -4.254413
stupid    -4.214097
no        -4.158478
worse     -4.061154
dull      -4.038338
poorly    -3.869834
minutes   -3.825656
script    -3.741286
even      -3.516839
waste of  -3.398161
annoying  -3.387961


                coef
definitely  3.093087
especially  3.100104
hilarious   3.181270
superb      3.182898
beautiful   3.309040
enjoyed     3.340268
highly      3.386042
love        3.609770
today       3.610295
well        3.693586
brilliant   3.807481
loved       3.890594
fun         3.898608
the best    3.923230
best        4.209893
amazing     4.265063
wonderful   4.276604
perfect     4.723809
excellent   5.782716
great       7.986098


## adicionar outros classificadores e também testar outras database!